In [ ]:
import math
import numpy as np
import random
import matplotlib.pyplot as plt
from matplotlib.pyplot import figure
import numba
from numba import njit
import random as rd
import time
import graphviz

import plotly.graph_objects as go
import numpy as np

from numba.typed import List
import copy
from scipy.cluster.vq import kmeans2
import matplotlib.pyplot as plt
import numpy as np
from sklearn.cluster import KMeans
from collections import defaultdict

In [ ]:
item_list = []
for i in range(200):
    a=rd.randint(40,60)
    b=rd.randint(40,80)
    c=rd.randint(40,60)

    item = np.array([a, b, c, 0, 0, 0], np.int_)
    item_list.append(item)
item_list= np.array(item_list, np.int_)
bin_size = np.array([500,500,500], np.int_)
PI_LENGTH = 0
PI_WIDTH = 1
PI_HEIGHT = 2
PI_X = 3
PI_Y = 4
PI_Z = 5

In [ ]:
def shelf(item_list, bin_size):
    pv_x = 0
    pv_y = 0
    pv_z = 0
    max_height = 0
    max_length = 0
    temp_list_id = [ ]
    all_list_id = [ ]
    direction=1
    for i in range(len(item_list)):
        if pv_z + item_list[i, PI_HEIGHT] > bin_size[PI_HEIGHT]:
            continue
        if pv_x + item_list[i, PI_LENGTH] > bin_size[PI_LENGTH]:
            pv_x = 0
            pv_y = 0
            pv_z += max_height
            max_height = 0
        if pv_y + item_list[i, PI_WIDTH] > bin_size[PI_WIDTH]:
            pv_x += max_height
            pv_y = 0
            max_lenght = 0
            temp_list_id.sort(key= lambda id: direction*item_list[id, PI_LENGTH])
            direction = -direction
        temp_list_id.append(i)
        item_list[i][PI_X] = pv_x
        item_list[i][PI_Y] = pv_y
        item_list[i][PI_Z] = pv_z
        pv_y += item_list[i,PI_WIDTH]
        max_length = max(max_length, item_list[i,PI_LENGTH])
        max_height = max(max_height, item_list[i,PI_HEIGHT])
    return item_list

In [ ]:
def check_over_lap(x1, a1, y1, b1, x2, a2, y2, b2):
    if x1 >= x2+a2 or x1+a1 <= x2:
        return False
    if y1 >= y2+b2 or y1+b1 <= y2:
        return False
    return True

In [ ]:
def shaking_items(item_list, invert, shaking_direction, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    if not invert:item_list = item_list[np.argsort(item_list[:,PI_LENGTH + shaking_direction] + item_list[:,PI_X + shaking_direction])]
    elif invert: item_list = item_list[np.argsort(-item_list[:,PI_LENGTH + shaking_direction] - item_list[:,PI_X + shaking_direction])]
    for i in range(len(item_list)):
        if item_list[i][PI_X] < shaking_zone[PI_X] or item_list[i][PI_X] > shaking_zone[PI_X] + shaking_zone[PI_LENGTH] or item_list[i][PI_Y] < shaking_zone[PI_Y] or item_list[i][PI_Y] > shaking_zone[PI_Y] + shaking_zone[PI_WIDTH] or item_list[i][PI_Z] < shaking_zone[PI_Z] or item_list[i][PI_Z] > shaking_zone[PI_Z] + shaking_zone[PI_HEIGHT]:
          continue
        if item_list[i][PI_X] >= static_zone[PI_X] and item_list[i][PI_X] <= static_zone[PI_X] + static_zone[PI_LENGTH] and item_list[i][PI_Y] >= static_zone[PI_Y] and item_list[i][PI_Y] <= static_zone[PI_Y] + static_zone[PI_WIDTH] and item_list[i][PI_Z] >= static_zone[PI_Z] and item_list[i][PI_Z] <= static_zone[PI_Z] + static_zone[PI_HEIGHT]:
          continue
        if not invert: new_pos=0
        elif invert:
            new_pos = bin_size[PI_LENGTH + shaking_direction]  - item_list[i][PI_LENGTH + shaking_direction]
        for j in range(i):
            a = (shaking_direction + 1)%3
            b = (shaking_direction + 2)%3
            x = PI_X + a
            y = PI_X + b
            if not invert:
              if check_over_lap(item_list[i, x], item_list[i, a], item_list[i, y], item_list[i, b], item_list[j, x], item_list[j, a], item_list[j, y], item_list[j, b]):
                  new_pos= max(item_list[j][PI_X + shaking_direction]+ item_list[j][PI_LENGTH + shaking_direction], new_pos)
            elif invert:
              if check_over_lap(item_list[i, x], item_list[i, a], item_list[i, y], item_list[i, b], item_list[j, x], item_list[j, a], item_list[j, y], item_list[j, b]):
                  new_pos= min(item_list[j][PI_X + shaking_direction]- item_list[i][PI_LENGTH + shaking_direction], new_pos)
        item_list[i][PI_X + shaking_direction]= new_pos
    return item_list

In [ ]:
def shaking_x(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 0, shaking_zone, static_zone)
    return item_list
def shaking_x_invert(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 0, shaking_zone, static_zone)
    return item_list
def shaking_y(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 1, shaking_zone, static_zone)
    return item_list
def shaking_y_invert(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 1, shaking_zone, static_zone)
    return item_list
def shaking_z(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 0, 2, shaking_zone, static_zone)
    return item_list
def shaking_z_invert(item_list, shaking_zone = np.array([10000000, 10000000, 10000000, 0, 0, 0]), static_zone = np.array([0, 0, 0, -1, -1, -1], np.int_)):
    item_list = shaking_items(item_list, 1, 2, shaking_zone, static_zone)
    return item_list

In [ ]:
def draw_packed_bin(item_list, bin_size):
    def get_random_color():
        return "#%06x" % rd.randint(0, 0xFFFFFF)
    color_list  =  []
    for i in range(1000):
        color_list.append(get_random_color())


    data  =  []

    data.append(
        go.Mesh3d(
            x = [0, 0, int(bin_size[0]), int(bin_size[0]), 0, 0, int(bin_size[0]), int(bin_size[0])],
            y = [0, int(bin_size[1]), int(bin_size[1]), 0, 0, int(bin_size[1]), int(bin_size[1]), 0],
            z = [0, 0, 0, 0, int(bin_size[2]), int(bin_size[2]), int(bin_size[2]), int(bin_size[2])],
            name = 'y',
            showscale = True,
            color = "#555",
            opacity = 0.1
        )
    )

    for index in range(len((item_list))):
        item =  item_list[index]
        ox_item  =  int(item[3])
        oy_item  =  int(item[4])
        oz_item  =  int(item[5])

        extreme_x  =  int(item[0]) + ox_item
        extreme_y  =  int(item[1]) + oy_item
        extreme_z  =  int(item[2]) + oz_item

        data.append(
            go.Mesh3d(
                x = [ox_item, ox_item, extreme_x, extreme_x, ox_item, ox_item, extreme_x, extreme_x],
                y = [oy_item, extreme_y, extreme_y, oy_item, oy_item, extreme_y, extreme_y, oy_item],
                z = [oz_item, oz_item, oz_item, oz_item, extreme_z, extreme_z, extreme_z, extreme_z],
                i  =  [7, 0, 0, 0, 4, 4, 6, 6, 4, 0, 3, 2],
                j  =  [3, 4, 1, 2, 5, 6, 5, 2, 0, 1, 6, 3],
                k  =  [0, 7, 2, 3, 6, 7, 1, 1, 5, 5, 7, 6],
                showscale  =  True,
                color  =  get_random_color(),
                hovertext  =  "box "+str(index),
                opacity = 1, flatshading  =  True
            )
        )
        data.append(go.Scatter3d(x = [0], y = [0], z = [0], mode  =  'markers'))

    fig  =  go.Figure(data)
    fig.update_layout(
        autosize = False,
        width = 1300,
        height  =  900,
        scene_aspectmode  =  'data',
        scene_camera  =  dict( eye = dict(x = -2, y = -2, z = -1))
        )

    fig.show()

In [ ]:
item_list = item_list[np.argsort(1000*item_list[:,PI_Z] + item_list[:,PI_X])]
item_list = shelf(item_list, bin_size)
item_list = shaking_x(item_list)
draw_packed_bin(item_list, bin_size)

In [ ]:
for _ in range(10):
    item_list = shaking_x_invert(item_list, np.array([10000000, 10000000, 10000000, 0, 0, 170]), np.array([0, 0, 0, -1, -1, -1], np.int_))
    item_list = shaking_y_invert(item_list, np.array([10000000, 10000000, 10000000, 0, 0, 170]), np.array([0, 0, 0, -1, -1, -1], np.int_))
    item_list = shaking_z(item_list, np.array([10000000, 10000000, 10000000, 0, 0, 170]), np.array([0, 0, 0, -1, -1, -1], np.int_))
item_list = shaking_x(item_list)
item_list = shaking_y(item_list)
draw_packed_bin(item_list, bin_size)